In [ ]:
# !pip install tensorflow==1.15

In [ ]:
import pandas as pd
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
tf.__version__

'1.15.0'

In [ ]:
def load_data():
    data = pd.read_csv('dataset.csv')
    features = data[['Age','Sex','BP','Cholesterol','Na_to_K']]
    label = data[['Drug']]

    return features, label

In [ ]:
features, label = load_data()

In [ ]:
transformer = ColumnTransformer([('transform', OneHotEncoder(), [1,2,3])])
features = np.array(transformer.fit_transform(features), dtype=np.float)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.2)

In [ ]:
scaler = MinMaxScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
encoder = OneHotEncoder().fit(y_train)
y_train = encoder.transform(y_train).toarray()
y_test = encoder.transform(y_test).toarray()

In [ ]:
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
y_train, y_test = y_train.astype('float32'), y_test.astype('float32')

In [ ]:
EPOCH = 4000
ALPHA = 0.01
n_features = x_train.shape[1]
n_classes = y_train.shape[1]

In [ ]:
input_tensor = tf.placeholder(tf.float32)
label_tensor = tf.placeholder(tf.float32)

In [ ]:
neurons = [8, 8, n_classes]
n_layers = len(neurons)

In [ ]:
parameters = {}
for i in range(n_layers):
  parameters[f'W{i+1}'] = tf.Variable(
      tf.random.normal([
          n_features if (i == 0) else neurons[i-1], neurons[i]
      ]),
      tf.float32
  )

  parameters[f'B{i+1}'] = tf.Variable(
      tf.random.normal(
          [1, neurons[i]]
      ),
      tf.float32
  )

In [ ]:
def forward():
  a = input_tensor
  for i in range(n_layers):
    w = parameters[f'W{i+1}']
    b = parameters[f'B{i+1}']

    z = tf.matmul(a, w) + b
    a = tf.nn.sigmoid(z)
  return a

In [ ]:
with tf.Session() as sess:
  logits_tensor = forward()
  loss_tensor = tf.reduce_mean(0.5 * (label_tensor - logits_tensor) ** 2)
  true_preds_tensor = tf.equal(tf.argmax(logits_tensor, axis=1), tf.argmax(label_tensor, axis=1))
  acc_tensor = tf.reduce_mean(tf.cast(true_preds_tensor, tf.float32))
  optimizer = tf.train.GradientDescentOptimizer(ALPHA).minimize(loss_tensor)

  sess.run(tf.global_variables_initializer())
  for i in range(1, EPOCH+1):
    sess.run(optimizer, feed_dict={
        input_tensor:x_train,
        label_tensor:y_train
    })
    loss = sess.run(loss_tensor, feed_dict={
        input_tensor:x_train,
        label_tensor:y_train
    })
    val_acc = sess.run(acc_tensor, feed_dict={
        input_tensor:x_test,
        label_tensor:y_test
    })

    if i % 200 == 0:
      print(f'Epoch {i} - Loss {loss:.7f}')
    if i == 4000:
      print(f'Evaluation Accuracy {(val_acc * 100):.2f}%')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 200 - Loss 0.2506754
Epoch 400 - Loss 0.2458012
Epoch 600 - Loss 0.2406672
Epoch 800 - Loss 0.2352954
Epoch 1000 - Loss 0.2296925
Epoch 1200 - Loss 0.2238283
Epoch 1400 - Loss 0.2176263
Epoch 1600 - Loss 0.2109869
Epoch 1800 - Loss 0.2038569
Epoch 2000 - Loss 0.1963353
Epoch 2200 - Loss 0.1887500
Epoch 2400 - Loss 0.1816005
Epoch 2600 - Loss 0.1753481
Epoch 2800 - Loss 0.1702246
Epoch 3000 - Loss 0.1662092
Epoch 3200 - Loss 0.1631346
Epoch 3400 - Loss 0.1607954
Epoch 3600 - Loss 0.1590081
Epoch 3800 - Loss 0.1576281
Epoch 4000 - Loss 0.1565482
Evaluation Accuracy 22.50%
